In [ ]:
"""
Vacuum Cleaner Agent-Based Simulation
Modelo multiagente que simula agentes aspiradores limpiando celdas sucias en el grid.
Implementación: MESA 3.3.1 (Multi-Agent Modeling in Python)

Autor:  Sebastián Álvarez Fuentes - A01800280 
        Isaac Calderon Laflor - A01751722
        Miguel Angel Argumedo - A01751322
Fecha: 2025-11-13
"""

# Imports de librerías estándar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Imports de ipywidgets para interfaz interactiva
from ipywidgets import IntSlider, FloatSlider, Button, Output, VBox, HBox
from IPython.display import clear_output

# Imports de MESA para modelado multiagente
import mesa
from mesa.discrete_space import CellAgent, OrthogonalMooreGrid


In [ ]:
"""
Clase VacuumAgent: Agente aspirador que limpia celdas sucias.

Comportamiento:
- Limpia la celda actual si está sucia
- Se mueve a una celda vecina aleatoria si la celda actual está limpia
- Utiliza neighborhood de Moore (8 celdas vecinas)
"""

class VacuumAgent(CellAgent):
    """
    Agente aspirador que navega y limpia el grid.
    Hereda de CellAgent para acceso integrado a el grid.
    """

    def __init__(self, model, cell):
        """
        Inicializa un agente aspirador.
        
        Parámetros:
            model: Referencia al modelo principal
            cell: Celda inicial donde se coloca el agente
        """
        super().__init__(model)
        self.cell = cell
        self.moves = 0

    def move(self):
        """
        Mueve el agente a una celda vecina aleatoria si la celda actual está limpia.
        Si la celda actual está sucia, el agente permanece en su posición.
        Incrementa el contador de movimientos cuando se realiza un desplazamiento.
        """
        if self.cell.coordinate not in self.model.dirtyCells:
            neighbors = list(self.cell.neighborhood.select())
            if neighbors:
                self.cell = self.model.random.choice(neighbors)
                self.moves += 1

    def clean(self):
        """
        Limpia la celda actual si se encuentra en estado sucio.
        Remueve la celda de la lista de celdas sucias del modelo.
        """
        if self.cell.coordinate in self.model.dirtyCells:
            self.model.dirtyCells.remove(self.cell.coordinate)

    def step(self):
        """
        Ejecuta un paso del agente: primero limpia, luego se mueve.
        Se ejecuta una vez por iteración del modelo.
        """
        self.clean()
        self.move()


In [ ]:
"""
Clase VacuumModel: Modelo principal que simula el entorno de limpieza.

El modelo mantiene:
- el grid con Moore neighborhood (8 vecinos)
- Un conjunto de celdas sucias en posiciones aleatorias
- Agentes aspiradores que inician en posición [1,1]
- Recopilación de datos para análisis estadístico
"""

class VacuumModel(mesa.Model):
    """
    Modelo que simula agentes aspiradores limpiando un grid bidimensional.
    Implementa lógica de simulación, recolección de datos y condiciones de parada.
    """

    def __init__(
        self,
        numAgents=2,
        width=10,
        height=10,
        dirtyPercentage=20,
        maxSteps=100,
        seed=None,
    ):
        """
        Inicializa el modelo con los parámetros especificados.
        
        Parámetros:
            numAgents: Número de agentes aspiradores a crear
            width: Ancho del grid
            height: Alto del grid
            dirtyPercentage: Porcentaje de celdas sucias (0-100)
            maxSteps: Máximo número de pasos de simulación
            seed: Semilla para reproducibilidad (None para aleatorio)
        """
        super().__init__(seed=seed)

        self.numAgents = numAgents
        self.width = width
        self.height = height
        self.dirtyPercentage = dirtyPercentage
        self.maxSteps = maxSteps

        # Crear grilla con vecindario Moore (8 vecinos)
        self.grid = OrthogonalMooreGrid(
            (width, height), torus=False, capacity=10, random=self.random
        )

        # Inicializar celdas sucias en posiciones aleatorias
        totalCells = width * height
        numDirty = int(totalCells * dirtyPercentage / 100)
        self.dirtyCells = set()

        while len(self.dirtyCells) < numDirty:
            x = self.random.randrange(width)
            y = self.random.randrange(height)
            self.dirtyCells.add((x, y))

        # Crear agentes y colocarlos todos en celda [1,1]
        startCell = self.grid[1, 1]
        agentCells = [startCell] * numAgents
        VacuumAgent.create_agents(self, numAgents, agentCells)

        # Inicializar recolector de datos
        self.datacollector = mesa.DataCollector(
            model_reporters={
                "Clean Percentage": self.computeCleanPercentage,
                "Dirty Cells": lambda m: len(m.dirtyCells),
            },
            agent_reporters={"Moves": "moves"},
        )
        self.running = True

    def computeCleanPercentage(self):
        """
        Calcula el porcentaje de celdas limpias en el grid.
        
        Retorna:
            Porcentaje de celdas limpias (0.0 a 100.0)
        """
        totalCells = self.width * self.height
        cleanCells = totalCells - len(self.dirtyCells)
        return (cleanCells / totalCells) * 100.0

    def step(self):
        """
        Ejecuta un paso de la simulación.
        Recolecta datos, ejecuta paso de todos los agentes y verifica condiciones de parada.
        """
        self.datacollector.collect(self)
        self.agents.shuffle_do("step")

        # Condición de parada: todas las celdas limpias o máximo de pasos alcanzado
        if len(self.dirtyCells) == 0 or self.steps >= self.maxSteps:
            self.running = False


In [ ]:
"""
Clase InteractiveVacuumSimulation: Interfaz interactiva para la simulación.

Proporciona:
- Visualización del grid en tiempo real
- Control manual de pasos de simulación
- Creación y reinicio de modelos con parámetros dinámicos
"""

class InteractiveVacuumSimulation:
    """
    Controlador interactivo para la simulación de aspiradores.
    Gestiona creación de modelos, visualización y ejecución de pasos.
    """

    def __init__(self):
        """
        Inicializa la simulación interactiva con variables nulas.
        Todas las variables se inicializan explícitamente.
        """
        self.model = None
        self.stepCount = 0
        self.isRunning = False

    def createModel(self, numAgents, width, height, dirtyPercentage, maxSteps):
        """
        Crea un nuevo modelo con los parámetros especificados.
        Reinicia los contadores internos de la simulación.
        
        Parámetros:
            numAgents: Número de agentes aspiradores
            width: Ancho del grid
            height: Alto del grid
            dirtyPercentage: Porcentaje inicial de celdas sucias
            maxSteps: Límite máximo de pasos
        """
        self.model = VacuumModel(
            numAgents=numAgents,
            width=width,
            height=height,
            dirtyPercentage=dirtyPercentage,
            maxSteps=maxSteps,
            seed=42
        )
        self.stepCount = 0
        self.isRunning = True
        self.model.datacollector.collect(self.model)

    def drawGrid(self):
        """
        Dibuja el estado actual de el grid en una figura de matplotlib.
        Muestra celdas limpias (blanco), celdas sucias (café) y agentes (azul).
        Incluye información del paso, porcentaje limpio y celdas sucias restantes.
        """
        if self.model is None:
            return

        plt.close('all')
        fig, ax = plt.subplots(figsize=(8.0, 8.0))

        # Dibujar líneas de grilla
        for i in range(self.model.width + 1):
            ax.axvline(i - 0.5, color='lightgray', linewidth=0.5)
        for i in range(self.model.height + 1):
            ax.axhline(i - 0.5, color='lightgray', linewidth=0.5)

        # Dibujar celdas sucias
        for (x, y) in self.model.dirtyCells:
            rect = plt.Rectangle((x - 0.5, y - 0.5), 1.0, 1.0,
                                facecolor='brown', alpha=0.5, edgecolor='none')
            ax.add_patch(rect)

        # Dibujar agentes
        for agent in self.model.agents:
            ax.scatter(agent.cell.coordinate[0], agent.cell.coordinate[1],
                      s=200, color='tab:blue', zorder=10, edgecolors='darkblue', linewidth=2)

        # Configurar límites y etiquetas
        ax.set_xlim(-1, self.model.width)
        ax.set_ylim(-1, self.model.height)
        ax.set_aspect('equal')
        ax.invert_yaxis()
        ax.set_xlabel('X')
        ax.set_ylabel('Y')

        cleanPercent = self.model.computeCleanPercentage()
        dirtyCount = len(self.model.dirtyCells)
        ax.set_title(f'Vacuum Cleaner - Step: {self.stepCount} | Clean: {cleanPercent:.1f}% | Dirty: {dirtyCount}')

        plt.tight_layout()
        plt.show()

    def stepSimulation(self, numSteps=1):
        """
        Ejecuta la simulación por numSteps iteraciones.
        Avanza los agentes y recolecta datos en cada paso.
        Verifica condiciones de parada después de cada paso.
        
        Parámetros:
            numSteps: Número de pasos a ejecutar (por defecto 1)
        """
        if self.model is None:
            return

        for _ in range(numSteps):
            if self.model.running:
                self.model.agents.shuffle_do("step")
                self.model.datacollector.collect(self.model)
                self.stepCount += 1

                if len(self.model.dirtyCells) == 0 or self.stepCount >= self.model.maxSteps:
                    self.model.running = False
            else:
                self.isRunning = False
                break

# Crear instancia de la simulación interactiva
sim = InteractiveVacuumSimulation()

# Crear controles deslizantes para parámetros
numAgentsSlider = IntSlider(value=2, min=1, max=10, step=1, description='Agents:')
widthSlider = IntSlider(value=10, min=5, max=30, step=1, description='Width:')
heightSlider = IntSlider(value=10, min=5, max=30, step=1, description='Height:')
dirtySlider = FloatSlider(value=30.0, min=5.0, max=80.0, step=5.0, description='Dirty %:')
maxStepsSlider = IntSlider(value=200, min=50, max=500, step=50, description='Max Steps:')

# Crear botones de control
resetButton = Button(description='Reset Simulation', button_style='warning')
stepButton = Button(description='Step (1x)', button_style='info')
step10Button = Button(description='Step (10x)', button_style='info')
runButton = Button(description='Run to End', button_style='success')

# Área de salida para visualización
outputArea = Output()

def onResetClicked(button):
    """
    Manejador para botón de reinicio.
    Crea un nuevo modelo con parámetros actuales de controles.
    Dibuja el grid inicial.
    
    Parámetros:
        button: Objeto de botón que activó el evento
    """
    with outputArea:
        clear_output(wait=True)
        sim.createModel(
            numAgents=int(numAgentsSlider.value),
            width=int(widthSlider.value),
            height=int(heightSlider.value),
            dirtyPercentage=int(dirtySlider.value),
            maxSteps=int(maxStepsSlider.value)
        )
        print(f"Simulation reset with {numAgentsSlider.value} agents on {widthSlider.value}x{heightSlider.value} grid")
        print(f"Step: {sim.stepCount} | Dirty cells: {len(sim.model.dirtyCells)} ({sim.model.dirtyPercentage}%)")
        sim.drawGrid()

def onStepClicked(button):
    """
    Manejador para botón de avance de un paso.
    Ejecuta un paso y redibuja el grid.
    
    Parámetros:
        button: Objeto de botón que activó el evento
    """
    with outputArea:
        if sim.model is None:
            print("Please reset simulation first")
        else:
            clear_output(wait=True)
            sim.stepSimulation(1)
            sim.drawGrid()
            print(f"Step: {sim.stepCount}")

def onStep10Clicked(button):
    """
    Manejador para botón de avance de diez pasos.
    Ejecuta diez pasos y redibuja el grid.
    
    Parámetros:
        button: Objeto de botón que activó el evento
    """
    with outputArea:
        if sim.model is None:
            print("Please reset simulation first")
        else:
            clear_output(wait=True)
            sim.stepSimulation(10)
            sim.drawGrid()
            print(f"Step: {sim.stepCount}")

def onRunClicked(button):
    """
    Manejador para botón de ejecución hasta el final.
    Ejecuta los pasos restantes hasta maxSteps o hasta limpiar todas las celdas.
    Muestra resultado final.
    
    Parámetros:
        button: Objeto de botón que activó el evento
    """
    with outputArea:
        if sim.model is None:
            print("Please reset simulation first")
        else:
            clear_output(wait=True)
            remaining = sim.model.maxSteps - sim.stepCount
            sim.stepSimulation(remaining)
            sim.drawGrid()
            if not sim.model.running:
                if len(sim.model.dirtyCells) == 0:
                    print(f"All cells cleaned in {sim.stepCount} steps")
                else:
                    print(f"Simulation completed in {sim.stepCount} steps")
                print(f"Final clean percentage: {sim.model.computeCleanPercentage():.1f}%")

# Vincular manejadores a botones
resetButton.on_click(onResetClicked)
stepButton.on_click(onStepClicked)
step10Button.on_click(onStep10Clicked)
runButton.on_click(onRunClicked)

# Organizar interfaz en layout
paramsBox = VBox([numAgentsSlider, widthSlider, heightSlider, dirtySlider, maxStepsSlider])
buttonsBox = VBox([resetButton, HBox([stepButton, step10Button]), runButton])
interface = HBox([paramsBox, buttonsBox])

print("Instructions:")
print("1. Adjust parameters using the sliders")
print("2. Click 'Reset Simulation' to create a new model")
print("3. Use Step buttons to advance the simulation")
print("4. Click 'Run to End' to complete the simulation\n")

display(interface)
display(outputArea)
